In [14]:
pip install ucimlrepo

SyntaxError: invalid syntax (1092723103.py, line 1)

In [8]:
# ------- Import Dataset -------------------

from ucimlrepo import fetch_ucirepo

# fetch dataset
air_quality = fetch_ucirepo(id=360)

# data (as pandas dataframes)
X = air_quality.data.features
y = air_quality.data.targets # unused?

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Combine Date and Time into a single DateTime column -> only 14 features
import pandas as pd

X['DateTime'] = pd.to_datetime(X['Date'] + ' ' + X['Time']) # Format (JJJJ-MM-DD HH:MM:SS, e.g.: 2004-03-10 18:00:00)
X = X.set_index('DateTime')
X.drop(['Date', 'Time'], axis=1, inplace=True)  # Remove the original Date and Time columns
X.replace(-200, 0, inplace=True)
X.replace(-200.0, 0, inplace=True)

In [ ]:
pd.set_option('display.width', 1000)
print(X.head())

In [ ]:
!nvidia-smi

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Assuming df is your DataFrame
# Set your target feature and predictors
target = 'target_feature'  # Replace with your target feature's name
predictors = df.columns[df.columns != target]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(df[predictors], df[target], test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")